# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ndibene dahra,15.3338,-15.4766,21.81,10,0,4.79,SN,1738792499
1,1,valparaiso,-33.0393,-71.6273,28.13,37,0,10.80,CL,1738792502
2,2,corner brook,48.9667,-57.9484,-13.20,90,100,5.16,CA,1738792503
3,3,constitucion,-35.3333,-72.4167,20.55,54,11,7.17,CL,1738792504
4,4,lebu,-37.6167,-73.6500,18.46,81,1,4.78,CL,1738792505


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot

city_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color = "Country"
)

# Display the map 

city_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [x,y]   (Country,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
pref_temp_max = city_data_df.loc[city_data_df["Max Temp"]<27]
pref_temp_min = pref_temp_max[pref_temp_max["Max Temp"]>21]
pref_wind_speed_max = pref_temp_min.loc[pref_temp_min["Wind Speed"] < 4.5]
pref_cloudiness = pref_wind_speed_max[pref_wind_speed_max["Cloudiness"]==0]

# Drop any rows with null values
pref_city = pref_cloudiness.dropna()

# Display sample data
pref_city

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
51,51,santa terezinha,-29.9002,-50.0904,26.63,88,0,4.48,BR,1738792562
56,56,batabano,22.7164,-82.2881,26.50,60,0,0.81,CU,1738792568
108,108,pipri,19.8000,75.5333,21.32,73,0,1.03,IN,1738792630
387,387,ringim,12.1548,9.1619,21.16,10,0,3.69,NG,1738792977
449,449,chokwe,-24.5333,32.9833,25.14,69,0,3.12,MZ,1738793053
462,462,massawa,15.6097,39.4500,26.02,69,0,2.65,ER,1738793070
552,552,mazatlan,23.2167,-106.4167,26.98,62,0,3.60,MX,1738792957
570,570,port blair,11.6667,92.7500,23.99,84,0,3.77,IN,1738793207


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = pref_city.loc[:,["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
51,santa terezinha,BR,-29.9002,-50.0904,88,
56,batabano,CU,22.7164,-82.2881,60,
108,pipri,IN,19.8000,75.5333,73,
387,ringim,NG,12.1548,9.1619,10,
449,chokwe,MZ,-24.5333,32.9833,69,
462,massawa,ER,15.6097,39.4500,69,
552,mazatlan,MX,23.2167,-106.4167,62,
570,port blair,IN,11.6667,92.7500,84,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "api_key": geoapify_key,
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    print(name_address)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
<Response [200]>
santa terezinha - nearest hotel: Pousada Santa Terezinha
<Response [200]>
batabano - nearest hotel: No hotel found
<Response [200]>
pipri - nearest hotel: No hotel found
<Response [200]>
ringim - nearest hotel: No hotel found
<Response [200]>
chokwe - nearest hotel: Limpopo Hotel
<Response [200]>
massawa - nearest hotel: Seghen International Hotel فندق دولي سيغن
<Response [200]>
mazatlan - nearest hotel: Mona Inn
<Response [200]>
port blair - nearest hotel: Andaman Ocean Breeze


,City,Country,Lat,Lng,Humidity,Hotel Name
51,santa terezinha,BR,-29.9002,-50.0904,88,Pousada Santa Terezinha
56,batabano,CU,22.7164,-82.2881,60,No hotel found
108,pipri,IN,19.8000,75.5333,73,No hotel found
387,ringim,NG,12.1548,9.1619,10,No hotel found
449,chokwe,MZ,-24.5333,32.9833,69,Limpopo Hotel
462,massawa,ER,15.6097,39.4500,69,Seghen International Hotel فندق دولي سيغن
552,mazatlan,MX,23.2167,-106.4167,62,Mona Inn
570,port blair,IN,11.6667,92.7500,84,Andaman Ocean Breeze


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [x,y]   (City,Humidity,Hotel Name,Country)